In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader

from sklearn.model_selection import KFold

# <center>Pré-processamento dos dados</center>

In [2]:
transform_dataset = transforms.Compose(
    [transforms.Resize(size = (224,224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5], std=[0.5])
    ]
)

dataset = datasets.ImageFolder('dataset_teste2', transform = transform_dataset)

In [3]:
dataset

Dataset ImageFolder
    Number of datapoints: 1000
    Root location: dataset_teste2
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.5], std=[0.5])
           )

In [6]:
dataset.class_to_idx

{'NORMAL': 0, 'PNEUMONIA': 1, 'TUBERCULOSE': 2}

# <center>Implementação da arquitetura CNN</center>

In [7]:
model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
    
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
    
    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
    
    nn.Flatten(),
    
    nn.Linear(128 * 28 * 28, 256),
    nn.Dropout(p=0.3),
    nn.ReLU(),

    nn.Linear(256, 128),
    nn.Dropout(p=0.3),
    nn.ReLU(),
    
    nn.Linear(128, 64),
    nn.Dropout(p=0.3),
    nn.ReLU(),
    
    nn.Linear(64, 3)
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo utilizado: {device}")

Dispositivo utilizado: cuda


In [8]:
model.to(device)

Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU()
  (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): Flatten(start_dim=1, end_dim=-1)
  (13): Linear(in_features=100352, out_features=256, bias=True)
  (14): Dropout(p=0.3, inplace=False)
  (15): ReLU()
  (16): Linear(in_features=256, out_features=128, bias=True

In [11]:
num_epoch = 250
learning_rate = 0.001
k_folds = 10

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
kf = KFold(n_splits=k_folds)

In [12]:
results = {}
pred_correta_treino = 0 
total_treino = 0

for fold, (train_idx, test_idx) in enumerate(kf.split(dataset)):
    print(f'Fold {fold+1}/{k_folds}')

    train_subset = Subset(dataset, train_idx)
    test_subset = Subset(dataset, test_idx)

    trainloader = DataLoader(train_subset, batch_size=16, shuffle=True)
    testloader = DataLoader(test_subset, batch_size=16, shuffle=False)

    for epoch in range(num_epoch):
        model.train()
        running_train_loss = 0.0

        for inputs, labels in trainloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
    
            optimizer.zero_grad()
            outputs = model(inputs)

            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item()

        train_loss = running_train_loss / len(trainloader)
        _, predicted = torch.max(outputs.data, 1)

        total_treino += labels.size(0)
        pred_correta_treino += (predicted == labels).sum().item()

        acc_treino = 100.0 * pred_correta_treino / total_treino

        print(f"Época {epoch + 1}/{num_epoch} - Perda no treinamento: {train_loss:.6f} - Acurácia: {acc_treino:.4f}")

    model.eval()
    pred_correta_teste = 0
    total_teste = 0

    with torch.no_grad():
        for images, labels in testloader:
            images = images.to(device)
            labels = labels.to(device)
        
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total_teste += labels.size(0)
            pred_correta_teste += (predicted == labels).sum().item()
    
    acc_teste = 100.0 * pred_correta_teste / total_teste
    print(f'Acc para o Fold {fold+1}: {acc_teste:.2f}%')
    results[fold] = acc_teste

    print("!!!Teste finalizado!!!")

print(f'\nResultados K-Fold: {results}')
print(f'Média Acc: {sum(results.values()) / k_folds:.2f}%')

Fold 1/10
Época 1/250 - Perda no treinamento: 1.461303 - Acurácia: 100.0000
Época 2/250 - Perda no treinamento: 0.422268 - Acurácia: 87.5000
Época 3/250 - Perda no treinamento: 0.288317 - Acurácia: 91.6667
Época 4/250 - Perda no treinamento: 0.286502 - Acurácia: 93.7500
Época 5/250 - Perda no treinamento: 0.167850 - Acurácia: 95.0000
Época 6/250 - Perda no treinamento: 0.103725 - Acurácia: 95.8333
Época 7/250 - Perda no treinamento: 0.121798 - Acurácia: 96.4286
Época 8/250 - Perda no treinamento: 0.098489 - Acurácia: 93.7500
Época 9/250 - Perda no treinamento: 0.110988 - Acurácia: 91.6667
Época 10/250 - Perda no treinamento: 0.104132 - Acurácia: 92.5000
Época 11/250 - Perda no treinamento: 0.061076 - Acurácia: 93.1818
Época 12/250 - Perda no treinamento: 0.095014 - Acurácia: 93.7500
Época 13/250 - Perda no treinamento: 0.092020 - Acurácia: 94.2308
Época 14/250 - Perda no treinamento: 0.105621 - Acurácia: 94.6429
Época 15/250 - Perda no treinamento: 0.072858 - Acurácia: 95.0000
Época 16

Tempo de execução: 12 horas